In [3]:
#!/usr/bin/env python
# coding: utf-8

import glob
import os
import csv
import copy
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("darkgrid")
fig = plt.figure()

#os.chdir("./CSV_Visualizer_python")
#os.chdir("../")
#実行ディレクトリを取得
dir_csv_viz = os.getcwd()
#dir_csv_viz = "C:\\Users\\administrator\\CSVDatasetMaker"
#入力ディレクトリ名を指定
dir_input = dir_csv_viz + "\\input"
#出力ディレクトリ名を指定
dir_output =dir_csv_viz + "\\output"
#使わないファイル用ディレクトリ名を指定
dir_nouse = dir_csv_viz + "\\nouse"

rename = input("rename?[no = 0, from configfile = 1, or write by yourself]: ")

PATH = os.getcwd()
obj_dir = "C:\\Users\\administrator\\Desktop\\MonitoringSystem(物体操作シーン解析)\\img\\20200116_223630\\10frame_sampling"
dirs = glob.glob(obj_dir+"\\*-*")
hpdirs = []
base_dirs = []
for d in dirs:
    hpdir = glob.glob(d+"\\hand_points\\mm")
    if hpdir:
        hpdirs.append(hpdir)
for hpd in hpdirs:
    base_dirs.append(hpd[0].split("\\hand_points\\mm")[0])
for base in base_dirs:
    inputs = glob.glob(base+"\\hand_points\\mm\\*.csv") 
    dir_output = base+"\\dataset3d"
#print(os.getcwd())
#print("Current Directory is : " + dir_csv_viz)
#print("Input Directory is : " + dir_input)
#print("Output Directory is : " + dir_output)

#取得するCSVファイルの名前規則設定ファイルを読み込み
#fp_ftype_conf = open(dir_csv_viz + "\\configFileType.txt", "r")
#file_type = ""
#for line in fp_ftype_conf:
#    file_type = line
#fp_ftype_conf.close()
#print(file_type)

#データセット用にリネームするかどうか決める(しない場合はそのまま)
    #rename = input("rename?[no = 0, from configfile = 1, or write by yourself]: ")

    #入力ディレクトリ内CSVファイルの名前リストを取得
    #csv_name_list = glob.glob(dir_input + "\\" + file_type)
    csv_name_list = inputs
    #print(csv_name_list)

    os.makedirs(dir_output + "\\data", exist_ok=True)
    os.makedirs(dir_output + "\\datac", exist_ok=True)
    os.makedirs(dir_output + "\\label", exist_ok=True)
    os.makedirs(dir_output + "\\labelc", exist_ok = True)
    os.makedirs(dir_output + "\\nouse", exist_ok = True)
    os.makedirs(dir_output + "\\all", exist_ok = True)
    os.makedirs(dir_output + "\\all\\label", exist_ok = True)
    os.makedirs(dir_output + "\\all\\labelc", exist_ok = True)

    #CSVファイル名リストのファイルを1つずつ可視化していく
    serial_num = 0
    lencsv = len(csv_name_list)
    for f in csv_name_list:
        fp_csv = open(f, "r")

        fname = ""
        #出力ファイル名の生成
        if rename == '0':
            fname = str(copy.copy(f))
            fname = fname.split("mm\\")[1]
            fname = fname.replace(".csv","")
            fname = fname + "_" + str(serial_num).zfill(4) ##ここは連番のために0埋めフォーマットにする必要あり(未実装)

        elif rename == '1':
            #リネームに関する設定ファイルを読み込み
            fp_rename_conf = open(dir_csv_viz + "\\configRename.txt", "r")
            rename = ""
            for line in fp_rename_conf:
                rename = line
            fname = rename + "_" + str(serial_num).zfill(4)
            fp_rename_conf.close()
        else:
            fname = rename + "_" + str(serial_num).zfill(4)
        #ax.set_title(fname)
        #fname = fname.replace(".csv", ".png")
        #print("fname: " + fname)

        #1つのCSVからデータを読み取る
        reader = csv.reader(fp_csv)
        data = [row for row in reader]
        data_T = [list(t) for t in zip(*data)]
        #print(data)
        #print(data_T)

        #id,x,y,confidency,zそれぞれの値列として取得
        ids = [int(id) for id in data_T[0]]
        x_values = [float(x) for x in data_T[1]]
        #y_values = [-1 * int(y) for y in data_T[2]]
        y_values = [float(y) for y in data_T[2]]
        c_values = [float(c) for c in data_T[3]]
        z_values = [float(z) for z in data_T[4]]
        #print(ids)
        #print(x_values)
        #print(y_values)
        #print(confidences)
        #print(z_values)

        #x,y,zのみを関節点ID0～45まで順番に並べたCSVを生成する
        buffer_t = []
        buffer_tc = []
        buffer_l = []
        buffer_lc = []
        for i in range(len(x_values)):
            buffer_tc.append(x_values[i])
            buffer_tc.append(y_values[i])
            buffer_tc.append(z_values[i])
            buffer_tc.append(c_values[i])
        buffer_tc.append(serial_num)
        for i in range(len(x_values)):
            buffer_t.append(x_values[i])
            buffer_t.append(y_values[i])
            buffer_t.append(z_values[i])
        buffer_t.append(serial_num)
        for i in range(len(x_values)):
            buffer_l.append(x_values[i])
            buffer_l.append(y_values[i])
            buffer_l.append(z_values[i])
        for i in range(len(x_values)):
            buffer_lc.append(x_values[i])
            buffer_lc.append(y_values[i])
            buffer_lc.append(z_values[i])
            buffer_lc.append(c_values[i])
        if serial_num == lencsv - 1:
            fpath_nut = dir_output + "\\nouse\\dataset_" + fname + "_t.csv"
            fpath_nutc = dir_output + "\\nouse\\dataset_" + fname + "_tc.csv"
            fpath_l = dir_output + "\\label\\dataset_" + fname + ".csv"
            fpath_lc = dir_output + "\\labelc\\dataset_" + fname + ".csv"
            fpath_all = dir_output + "\\all\\label\\dataset_"+fname+".csv"
            fpath_allc = dir_output + "\\all\\labelc\\dataset_"+fname+".csv"
            fp_out_nut = open(fpath_nut, "w")
            fp_out_nutc = open(fpath_nutc, "w")
            fp_out_l = open(fpath_l, "w")
            fp_out_lc = open(fpath_lc, "w")
            fp_out_a = open(fpath_all, "w")
            fp_out_ac = open(fpath_allc, "w")
            writer_nut = csv.writer(fp_out_nut)
            writer_nutc = csv.writer(fp_out_nutc)
            writer_l = csv.writer(fp_out_l)
            writer_lc = csv.writer(fp_out_lc)
            writer_a = csv.writer(fp_out_a)
            writer_ac = csv.writer(fp_out_ac)
            writer_nut.writerow(buffer_t)
            writer_nutc.writerow(buffer_tc)
            writer_l.writerow(buffer_l)
            writer_lc.writerow(buffer_lc)
            writer_a.writerow(buffer_l)
            writer_ac.writerow(buffer_lc)
            fp_out_nut.close()
            fp_out_nutc.close()
            fp_out_l.close()
            fp_out_lc.close()
            fp_out_a.close()
            fp_out_ac.close()
        elif serial_num == 0:
            fpath_nul = dir_output + "\\nouse\\dataset_" + fname + "_l.csv"
            fpath_nulc = dir_output + "\\nouse\\dataset_" + fname + "_lc.csv"
            fpath_t = dir_output + "\\data\\dataset_" + fname + ".csv"
            fpath_tc = dir_output + "\\datac\\dataset_" + fname + ".csv"
            fpath_all = dir_output + "\\all\\label\\dataset_"+fname+".csv"
            fpath_allc = dir_output + "\\all\\labelc\\dataset_"+fname+".csv"
            fp_out_nul = open(fpath_nul, "w")
            fp_out_nulc = open(fpath_nulc, "w")
            fp_out_t = open(fpath_t, "w")
            fp_out_tc = open(fpath_tc, "w")
            fp_out_a = open(fpath_all, "w")
            fp_out_ac = open(fpath_allc, "w")
            writer_nul = csv.writer(fp_out_nul)
            writer_nulc = csv.writer(fp_out_nulc)
            writer_t = csv.writer(fp_out_t)
            writer_tc = csv.writer(fp_out_tc)
            writer_a = csv.writer(fp_out_a)
            writer_ac = csv.writer(fp_out_ac)
            writer_nul.writerow(buffer_l)
            writer_nulc.writerow(buffer_lc)
            writer_t.writerow(buffer_t)
            writer_tc.writerow(buffer_tc)
            writer_a.writerow(buffer_l)
            writer_ac.writerow(buffer_lc)
            fp_out_nul.close()
            fp_out_nulc.close()
            fp_out_t.close()
            fp_out_tc.close()
            fp_out_a.close()
            fp_out_ac.close()
        else:
            fpath_t = dir_output + "\\data\\dataset_" + fname + ".csv"
            fpath_tc = dir_output + "\\datac\\dataset_" + fname + ".csv"
            fpath_l = dir_output + "\\label\\dataset_" + fname + ".csv"
            fpath_lc = dir_output + "\\labelc\\dataset_" + fname + ".csv"
            fpath_all = dir_output + "\\all\\label\\dataset_"+fname+".csv"
            fpath_allc = dir_output + "\\all\\labelc\\dataset_"+fname+".csv"
            #print(fpath)
            fp_out_t = open(fpath_t, "w")
            fp_out_tc = open(fpath_tc, "w")
            fp_out_l = open(fpath_l, "w")
            fp_out_lc = open(fpath_lc, "w")
            fp_out_a = open(fpath_all, "w")
            fp_out_ac = open(fpath_allc, "w")
            #fp_out = open(fpath, "w")
            writer_t = csv.writer(fp_out_t)
            writer_tc = csv.writer(fp_out_tc)
            writer_l = csv.writer(fp_out_l)
            writer_lc = csv.writer(fp_out_lc)
            writer_a = csv.writer(fp_out_a)
            writer_ac = csv.writer(fp_out_ac)

            writer_t.writerow(buffer_t)
            writer_tc.writerow(buffer_tc)
            writer_l.writerow(buffer_l)
            writer_lc.writerow(buffer_lc)
            writer_a.writerow(buffer_l)
            writer_ac.writerow(buffer_lc)

            fp_out_t.close()
            fp_out_tc.close()
            fp_out_l.close()
            fp_out_lc.close()
            fp_out_a.close()
            fp_out_ac.close()

        #出力画像ファイルパスを指定

        #保存ファイルパスの生成

        #plt.savefig(fpath)
        fp_csv.close()
        serial_num += 1
print("completed saving !")

rename?[no = 0, from configfile = 1, or write by yourself]: 0
completed saving !
